In [ ]:
import json
import re
import pandas as pd

In [ ]:
path =  "/mnt/sda2/datas/mbf_ir/data/vnexpress/chatgpt.txt"
with open(path, "r", encoding="utf-8") as f:
    contents = f.readlines()
    data = [json.loads(line) for line in contents]

In [ ]:
ir_data = []
for sample in data[1:]:
    passage = sample["context"]
    questions = sample["question"]
    
    # questions = re.sub("(QUESTION|Question|question)[\s_-]*\d*:*", "\n", questions)
    # questions = re.sub("(CÂU[\s_]HỎI|Câu[\s_]Hỏi|câu[\s_]hỏi)[\s_-]*\d*:*", "\n", questions)
    questions = re.sub("([Qq][uU][Ee][sS][Tt][Ii][Oo][Nn])[\s_-]*\d\s*:*", "\n", questions)
    questions = re.sub("([Cc][âÂ][Uu][\s_][Hh][Ỏỏ][Ii])[\s_-]*\d\s*:*", "\n", questions)

    
    questions = re.sub("\n+", "\n", questions)
    questions = [tmp.strip() for tmp in questions.split("\n") if len(tmp.strip()) > 0]
    if len(questions) <= 1:
        continue
    samples = [[question, passage] for question in questions]
    
    ir_data += samples

In [ ]:
df = pd.DataFrame(ir_data, columns=["question", "passage"])
df.head()

In [ ]:
def filter_question(question):
    _CHAR = "qwertyuiopadsfghjklzxcvbnmaăâáắấàằầảẳẩãẵẫạặậđeêéếèềẻểẽễẹệiíìỉĩịoôơóốớòồờỏổởõỗỡọộợuưúứùừủửũữụựyýỳỷỹỵ".upper()
    count = 0
    for i in _CHAR:
        if i in question:
            count += 1
        if count >= 2:
            return 1
    return 0

df["is_selected"] = df.question.apply(lambda x: filter_question(x))

In [ ]:
df[df.is_selected==1].to_csv("/mnt/sda2/datas/mbf_ir/data/vnexpress/question-context.txt")

### generate hard negative

In [ ]:
%cd /mnt/sda2/datas/mbf_ir/demo
import json
import os
from glob import glob
from tqdm import tqdm
from underthesea import sent_tokenize
from utils import norm_text
import pandas as pd
from bm25 import BM25

In [ ]:
df.head()

In [ ]:
passage = list(set(df["passage"].tolist()))
passages = pd.DataFrame(passage, columns=["passage_text"])
passages.head()

In [ ]:
bm25_model = BM25()
bm25_model.train(passages)

In [ ]:
query = """Trước chung kết Champions League mùa 2022-2023, Man City được đánh giá cao hơn hẳn Inter. Thống """
bm25_model.search(query=query, topk=5)

In [ ]:
def gen_hard_neg_smpl(passage):
    negs = bm25_model.search(query=query, topk=32)
    negs = [i[1] for i in negs]
    
    return negs

df["negatives"]=df.passage.apply(gen_hard_neg_smpl)

In [ ]:
df.head()

In [ ]:
msmacro_data = []
for index in tqdm(df.index):
    query = df["question"][index]
    positive = df["passage"][index]
    negatives = df["negatives"][index]
    
    passages = [
        {
            "passage_text":positive,
            "is_selected":1
        }
    ]
    
    for negative in negatives:
        sample = {
            "passage_text":negative,
            "is_selected":0
        }
        
        passages.append(sample)
    
    sample = {
        "query":query,
        "passages":passages
    }
    msmacro_data.append(sample)

In [ ]:
msmacro_data[0]

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")
path = "/mnt/sda2/datas/mbf_ir/data/vnexpress/vnexpress-dataset.jsonl"
save_data(path, msmacro_data)